In [1]:
from __future__ import division
import pandas as pd
import folium
import numpy as np
import matplotlib.cm as cm

In [2]:
daten_poly=pd.read_csv('Sickingmühlenbach_Ploy.csv', sep=';')
daten_poly.head(10)

,Point,Polygon Nr,Longitude,Latitude,Name,Pr-Wert
0,1,1,"7,11621","51,69787",Spot 1,300
1,2,1,"7,11614","51,69788",Spot 1,300
2,3,1,"7,11606","51,69794",Spot 1,300
3,4,1,"7,11616","51,69792",Spot 1,300
4,5,1,"7,11621","51,69787",Spot 1,300
5,6,2,"7,11657","51,69721",Spot 2,300
6,7,2,"7,11644","51,6972",Spot 2,300
7,8,2,"7,11657","51,69708",Spot 2,300
8,9,2,"7,11655","51,69711",Spot 2,300
9,10,2,"7,11657","51,69721",Spot 2,300


In [3]:
def komma2punkt(value):
    try:
        value = value.replace(',','.')
        return value
    except ValueError:
        return value

In [4]:
def polygon_df_to_geojson(df, properties, lat='Latitude', lon='Longitude'):
    """
    Turn a dataframe containing polygon (by "Polygon Nr") data into a geojson formatted python dictionary

    df : the dataframe to convert to geojson
    properties : a list of columns in the dataframe to turn into geojson feature properties
    lat : the name of the column in the dataframe that contains latitude data
    lon : the name of the column in the dataframe that contains longitude data
    """

    # find polygon groups
    group = df.groupby(['Polygon Nr']).groups.keys()

    # create a new python dict to contain our geojson data, using geojson format
    geojson = {"type": "FeatureCollection", "features": []}

    for i in range(len(group)):
        new_df = df.loc[df['Polygon Nr'] == [x for x in df.groupby(['Polygon Nr']).groups.keys()][i]]
        res_df = new_df.reset_index(drop=True)
        coords = []
        # create a feature template to fill in
        feature = {"type": "Feature",
                   "properties": {},
                   "geometry": {"type": "Polygon",
                                "coordinates": []}}
        # loop through each row in the dataframe and convert each row to geojson format
        for j in range(len(res_df)):
            coords.append([float(res_df[lon][j]), float(res_df[lat][j])])

        # fill in the coordinates
        feature["geometry"]["coordinates"] = [coords]

        # for each column, get the value and add it as a new feature property
        for prop in properties:
            feature['properties'][prop] = str(res_df[prop][i])
        
        # add this feature (aka, converted dataframe row) to the list of features inside our dict
        geojson["features"].append(feature)

    return geojson

In [23]:
def point_df_to_geojson(df, properties, lat='Latitude', lon='Longitude'):
    """
    Turn a dataframe containing point data into a geojson formatted python dictionary
    
    df : the dataframe to convert to geojson
    properties : a list of columns in the dataframe to turn into geojson feature properties
    lat : the name of the column in the dataframe that contains latitude data
    lon : the name of the column in the dataframe that contains longitude data
    """
    
    # create a new python dict to contain our geojson data, using geojson format
    geojson = {'type':'FeatureCollection', 'features':[]}

    # loop through each row in the dataframe and convert each row to geojson format
    for _, row in df.iterrows():
        # create a feature template to fill in
        feature = {'type':'Feature',
                   'properties':{},
                   'geometry':{'type':'Point',
                               'coordinates':[]}}

        # fill in the coordinates
        feature['geometry']['coordinates'] = [float(row[lon]),float(row[lat])]

        # for each column, get the value and add it as a new feature property
        for prop in properties:
            feature['properties'][prop] = row[prop]
        
        # add this feature (aka, converted dataframe row) to the list of features inside our dict
        geojson['features'].append(feature)
    
    return geojson

In [6]:
k2p_cols = ['Longitude','Latitude']
daten_poly[k2p_cols] = daten_poly[k2p_cols].applymap(lambda x: komma2punkt(x))
daten_poly

,Point,Polygon Nr,Longitude,Latitude,Name,Pr-Wert
0,1,1,7.11621,51.69787,Spot 1,300
1,2,1,7.11614,51.69788,Spot 1,300
2,3,1,7.11606,51.69794,Spot 1,300
3,4,1,7.11616,51.69792,Spot 1,300
4,5,1,7.11621,51.69787,Spot 1,300
5,6,2,7.11657,51.69721,Spot 2,300
6,7,2,7.11644,51.6972,Spot 2,300
7,8,2,7.11657,51.69708,Spot 2,300
8,9,2,7.11655,51.69711,Spot 2,300
9,10,2,7.11657,51.69721,Spot 2,300


In [7]:
#Latitude at 51.7 Degrees in m2
lat=111261.61
#Longitude at 51.7 Degrees in m2
lon=69136.13

In [8]:
from shapely.geometry import shape
pols = daten_poly.groupby(['Polygon Nr']).groups.keys()
k=[]
for i in pols:
    k.append(i)
area = []
for i in range(len(pols)):
    area.append((k[i],shape(polygon_df_to_geojson(daten_poly, ['Name','Pr-Wert'])['features'][i]['geometry']).area*lon*lat))

df = pd.DataFrame(area,columns=['Polygon Nr','Area'])
df

,Polygon Nr,Area
0,1,28.461129
1,2,54.999210
2,3,123.844374


In [9]:
lala=pd.merge(daten_poly, df, how='inner', on=['Polygon Nr'])
lala

,Point,Polygon Nr,Longitude,Latitude,Name,Pr-Wert,Area
0,1,1,7.11621,51.69787,Spot 1,300,28.461129
1,2,1,7.11614,51.69788,Spot 1,300,28.461129
2,3,1,7.11606,51.69794,Spot 1,300,28.461129
3,4,1,7.11616,51.69792,Spot 1,300,28.461129
4,5,1,7.11621,51.69787,Spot 1,300,28.461129
5,6,2,7.11657,51.69721,Spot 2,300,54.999210
6,7,2,7.11644,51.6972,Spot 2,300,54.999210
7,8,2,7.11657,51.69708,Spot 2,300,54.999210
8,9,2,7.11655,51.69711,Spot 2,300,54.999210
9,10,2,7.11657,51.69721,Spot 2,300,54.999210


In [10]:
keys = list(lala['Pr-Wert'].unique())
#color_range = list(np.linspace(0, 1, len(keys), endpoint=False))
#colors = [cm.tab20b(x) for x in color_range]
colors = ['red','green','yellow']
color_dict = dict(zip(keys, colors))

In [24]:
daten_points=pd.read_csv('Sickingmühlenbach_Points.csv', sep=';')
daten_points.head(10)

,Point,Longitude,Latitude,Name,Pr-Wert
0,1,"7,11615","51,69802","Spot am Pumpwerk, Schurf 1",300
1,2,"7,11661","51,69705",Schurf 2,0


In [25]:
daten_points[k2p_cols] = daten_points[k2p_cols].applymap(lambda x: komma2punkt(x))
daten_points

,Point,Longitude,Latitude,Name,Pr-Wert
0,1,7.11615,51.69802,"Spot am Pumpwerk, Schurf 1",300
1,2,7.11661,51.69705,Schurf 2,0


In [26]:
datensatz=polygon_df_to_geojson(lala, ['Name','Pr-Wert','Area'])
points=point_df_to_geojson(daten_points,['Name','Pr-Wert'])
m = folium.Map(
    location=[51.687,7.117],
    zoom_start=15,
    #tiles='Mapbox Bright'
)
folium.GeoJson(datensatz,
    style_function=lambda x :{
        'color' : 'red' if x['properties']['Pr-Wert']=='600' else 'orange' if x['properties']['Pr-Wert']=='300' else 'green',
        'opacity': 0.6,
        'fillColor':'red' if x['properties']['Pr-Wert']=='600' else 'orange' if x['properties']['Pr-Wert']=='300' else 'green'}
        ).add_to(m)
folium.GeoJson(points).add_to(m)
m

In [14]:
polygon_df_to_geojson(lala, ['Name','Pr-Wert','Area'])

{'type': 'FeatureCollection',
 'features': [{'type': 'Feature',
   'properties': {'Name': 'Spot 1',
    'Pr-Wert': '300',
    'Area': '28.461129395262944'},
   'geometry': {'type': 'Polygon',
    'coordinates': [[[7.11621, 51.69787],
      [7.11614, 51.69788],
      [7.11606, 51.69794],
      [7.11616, 51.69792],
      [7.11621, 51.69787]]]}},
  {'type': 'Feature',
   'properties': {'Name': 'Spot 2',
    'Pr-Wert': '300',
    'Area': '54.99920950028968'},
   'geometry': {'type': 'Polygon',
    'coordinates': [[[7.11657, 51.69721],
      [7.11644, 51.6972],
      [7.11657, 51.69708],
      [7.11655, 51.69711],
      [7.11657, 51.69721]]]}},
  {'type': 'Feature',
   'properties': {'Name': 'Spot 3',
    'Pr-Wert': '300',
    'Area': '123.84437383996887'},
   'geometry': {'type': 'Polygon',
    'coordinates': [[[7.11647, 51.69634],
      [7.11648, 51.69609],
      [7.11654, 51.69608],
      [7.11655, 51.69629],
      [7.11647, 51.69634]]]}}]}

In [15]:
def write_to_file(data,filename):
    f= open(filename,"w+")
    f.write(str(data).replace("'",'"'))
    f.close
    print('{} successfully written to {}'.format(data,filename))

In [21]:
write_to_file(datensatz,'Sickingmühlenbach.geojson')

{'type': 'FeatureCollection', 'features': [{'type': 'Feature', 'properties': {'Name': 'Spot 1', 'Pr-Wert': '300', 'Area': '28.461129395262944', 'style': {'color': 'orange', 'opacity': 0.6, 'fillColor': 'orange'}, 'highlight': {}}, 'geometry': {'type': 'Polygon', 'coordinates': [[[7.11621, 51.69787], [7.11614, 51.69788], [7.11606, 51.69794], [7.11616, 51.69792], [7.11621, 51.69787]]]}}, {'type': 'Feature', 'properties': {'Name': 'Spot 2', 'Pr-Wert': '300', 'Area': '54.99920950028968', 'style': {'color': 'orange', 'opacity': 0.6, 'fillColor': 'orange'}, 'highlight': {}}, 'geometry': {'type': 'Polygon', 'coordinates': [[[7.11657, 51.69721], [7.11644, 51.6972], [7.11657, 51.69708], [7.11655, 51.69711], [7.11657, 51.69721]]]}}, {'type': 'Feature', 'properties': {'Name': 'Spot 3', 'Pr-Wert': '300', 'Area': '123.84437383996887', 'style': {'color': 'orange', 'opacity': 0.6, 'fillColor': 'orange'}, 'highlight': {}}, 'geometry': {'type': 'Polygon', 'coordinates': [[[7.11647, 51.69634], [7.11648,

In [27]:
write_to_file(points,'Sickingmühlenbach_Points.geojson')

{'type': 'FeatureCollection', 'features': [{'type': 'Feature', 'properties': {'Name': 'Spot am Pumpwerk, Schurf 1', 'Pr-Wert': 300, 'style': {}, 'highlight': {}}, 'geometry': {'type': 'Point', 'coordinates': [7.11615, 51.69802]}}, {'type': 'Feature', 'properties': {'Name': 'Schurf 2', 'Pr-Wert': 0, 'style': {}, 'highlight': {}}, 'geometry': {'type': 'Point', 'coordinates': [7.11661, 51.69705]}}]} successfully written to Sickingmühlenbach_Points.geojson


In [ ]:
def poly_to_points(data,poly_nr,poly_col='Polygon Nr'):
    points = []
    for i in range(len(data)): 
        if data[poly_col][i] == poly_nr:
            points.append((float(data['Longitude'][i]),float(data['Latitude'][i])))
            
    #if points[-1] == points[0]:
        #points.pop()
    return points

In [ ]:
aaa=poly_to_points(daten_poly,1);aaa

In [ ]:
def poly_area(coords):
    t=0
    for count in range(len(coords)-1):
        y = coords[count+1][1] + coords[count][1]
        x = coords[count+1][0] - coords[count][0]
        z = y * x
        t += z
    return abs(t/2.0)

In [ ]:
ppp=poly_area(aaa);ppp

In [ ]:
from shapely.geometry import shape
sss=shape(polygon_df_to_geojson(daten_poly, ['Name','Pr-Wert'])['features'][0]['geometry']).area;sss

In [ ]:
#Latitude at 51.7 Degrees in m2
lat=111261.61
#Longitude at 51.7 Degrees in m2
lon=69136.13

In [ ]:
sss*lat*lon

In [ ]:
shape(polygon_df_to_geojson(daten_poly, ['Name','Pr-Wert'])['features'][1]['geometry']).area